In [1]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.tools import tool
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_core.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.output_parsers.pydantic import PydanticOutputParser
from langchain_core.language_models.llms import LLM
from langchain_core.output_parsers import JsonOutputParser
from snowflake.snowpark import Session
from typing import Any, List, Mapping, Optional
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_openai import ChatOpenAI
from langchain_mistralai.chat_models import ChatMistralAI
import pandas as pd
import json
from dotenv import load_dotenv
from typing import Optional
import os 
import tiktoken

load_dotenv()
SNOWFLAKE_PASSWORD = os.getenv("SNOWFLAKE_PASSWORD")
# OPENAI_API_KEY = os.getenv("OPEN_AI_API_KEY")
# MISTRAL_API_KEY = os.getenv("MISTRAL_SMALL_API_KEY")

In [2]:
connection_parameters = {
    "account": "khhmubi-uzb27668",
    "user": "JOSHUALESMU",
    "password": SNOWFLAKE_PASSWORD,
    "role": "ACCOUNTADMIN", 
    "warehouse": "COMPUTE_WH",  # optional
    } 
sp_session = Session.builder.configs(connection_parameters).create()  

In [3]:
def num_tokens_from_string(string: str, model_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model(model_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [4]:
class SnowflakeCortexLLM(LLM):

    sp_session: Session = None
    """Snowpark Session class instance, set before invoking the LLM to authenticate to an appropriate Snowflake account with Cortex LLMs provisioned."""

    model: str = 'mistral-7b'
    """The Snowflake cortex hosted LLM model name, default to `mistral-7b`. Refer to doc for other options."""

    cortex_function: str = 'complete'
    """The cortex function to use, defaulted to complete. for other types refer to doc"""

    llm_type: str = 'snowflake-cortex'
    """The type of LLM, defaulted to snowflake-cortex, for logging purposes only."""

    @property
    def _llm_type(self) -> str:
        return "snowflake_cortex"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        """Adapt the Snowflake Cortex LLM SQL-based API to this Python interface.
        Modify this accordingly to the available Snowflake Cortex LLM API.
        For example, this implementation is based on the following snowflake SQL command: 
        `SELECT SNOWFLAKE.CORTEX.COMPLETE('<model_name>', '<prompt_text>');`
        """    
        prompt_text = prompt
        # simple version
        # sql_statement = f'''select snowflake.cortex.{self.cortex_function}('{self.model}','{prompt_text}') as llm_reponse;'''
        # version with parameters and returns the token counts
        # use double {{}} to escape the curly braces in the f-string
        sql_statement = f""" 
            SELECT SNOWFLAKE.CORTEX.{self.cortex_function}
            (
                '{self.model}',
                [
                    {{
                        'role': 'user',
                        'content': '{prompt_text}'
                    }}
                ],
                {{
                    'temperature': 0
                }}
            )
            AS LLM_RESPONSE;            
            """
        l_rows = self.sp_session.sql(sql_statement).collect()
        llm_response = l_rows[0]['LLM_RESPONSE'] # only 1 row is expected from the SQL statement as it is applied to 1 prompt
        return llm_response

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {
            "model": self.model
            ,"cortex_function" : self.cortex_function
            ,"snowpark_session": self.sp_session.session_id
        }
    @property
    def _llm_type(cls) -> str:
        """Get the type of language model used by this chat model. Used for logging purposes only."""
        return cls.llm_type

sf_llm = SnowflakeCortexLLM(sp_session=sp_session)
print(sf_llm.model)

# set a different model from the default
sf_llm = SnowflakeCortexLLM(sp_session=sp_session, model='mixtral-8x7b')
print(sf_llm.model)

mistral-7b
mixtral-8x7b


In [9]:
# df = pd.read_csv('./inputs/sample_news_api_ml_100rows.csv')
# df_wkwebster = pd.read_csv('./inputs/sample_news_api_ml_wkwebster_2.csv')
# body_cleaned = df['BODY_CLEANED'].to_list()[:90] + df_wkwebster['BODY_CLEANED'].to_list()[:10]
# body_cleaned = [body.replace("'", "\\'") for body in body_cleaned]

## 1. Simple Examples

In [5]:
# Method 1: define the schema for custom tools using the @tool decorator on Python functions
@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b


tools = [add, multiply]

# Method 2: define the schema using Pydantic. Pydantic is useful when your tool inputs are more complex
class add(BaseModel):
    """Add two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


tools = [add, multiply]


In [51]:
# After defining the tools, we can bind them to the LLM:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
llm_with_tools = llm.bind_tools(tools)

- When you just use bind_tools(tools), the model can choose whether to return one tool call, multiple tool calls, or no tool calls at all.
- Some models support a tool_choice parameter that gives you some ability to force the model to call a tool.
- For models that support this, you can pass in the name of the tool you want the model to always call tool_choice="xyz_tool_name". 
- Or you can pass in tool_choice="any" to force the model to call at least one tool, without specifying which tool specifically.
    - Currently tool_choice="any" functionality is supported by OpenAI, MistralAI, FireworksAI, and Groq.
    - Currently Anthropic does not support tool_choice at all.

In [13]:
# # If we wanted our model to always call the multiply tool we could do:
# always_multiply_llm = llm.bind_tools([multiply], tool_choice="multiply")

# # And if we wanted it to always call at least one of add or multiply, we could do:
# always_call_tool_llm = llm.bind_tools([add, multiply], tool_choice="any")

If tool calls are included in a LLM response, they are attached to the corresponding AIMessage or AIMessageChunk (when streaming) as a list of ToolCall objects in the `.tool_calls` attribute.

In [7]:
query = "What is 3 * 12? Also, what is 11 + 49?"

llm_with_tools.invoke(query).tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_BwVtZ7PLznihXqtYQd5N0dlq'},
 {'name': 'add',
  'args': {'a': 11, 'b': 49},
  'id': 'call_yNpMPCrxqg1sSKIM2eVXoM06'}]

In [8]:
chain = llm_with_tools | PydanticToolsParser(tools=[multiply, add])
chain.invoke(query)

[multiply(a=3, b=12), add(a=11, b=49)]

In [9]:
# alternatively, the @tool decorator can be used instead of the Pydantic model

@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b


tools = [add, multiply]
llm_with_tools = llm.bind_tools(tools)

messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zTPCA6IJ8blahMONg3GKzcuv', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_QenQzEZrKCqW8qreQvLrDGbb', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 144, 'total_tokens': 193}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-83094e4f-aaa9-439d-a176-9c173e191093-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_zTPCA6IJ8blahMONg3GKzcuv'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_QenQzEZrKCqW8qreQvLrDGbb'}]),
 ToolMessage(content='36', tool_call_id='call_zTPCA6IJ8blahMONg3GKzcuv'),
 ToolMessage(content='60', tool_call_id='call_QenQzEZrKCqW8qreQvLrDGbb')]

In [10]:
llm_with_tools.invoke(messages)

AIMessage(content='3 * 12 is 36 and 11 + 49 is 60.', response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 209, 'total_tokens': 227}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6bb8e55a-6b7a-418d-af13-3b4855a5cfe1-0')

In [11]:
# Method 2: define the schema using Pydantic. Pydantic is useful when your tool inputs are more complex
class add(BaseModel):
    """Add two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


tools = [add, multiply]
llm_with_tools = llm.bind_tools(tools)
messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)
messages
# llm_with_tools.invoke(messages)

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_OWjEndPeN1Rt5NTzlqpjLu74', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_Bl7HBNKU1YqxbIiP5fOp8cmQ', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 105, 'total_tokens': 154}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-65f0d4d9-279f-41a4-b583-7f7bd273bb7c-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_OWjEndPeN1Rt5NTzlqpjLu74'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_Bl7HBNKU1YqxbIiP5fOp8cmQ'}])]

## 2 Extract Structured Output

### 0. Setup Pydantic model, tools, sample data and prompt templates

In [5]:
class NewsInfo(BaseModel):
    """Information extracted from the text."""
    title: str = Field(
        description="One sentence summary of the article of maximum 200 characters, prefereably with the event, location and time information."
    )
    summary: str = Field(
        description="A short summary of the text, maximum 200 words"
    )
    impact: str = Field( 
        description="Answer only Yes or No to this question: does this event negatively impact a supply chain network (the movement of people and goods)? Answer this by following the following reasoning steps: \
            If the event can directly impact a supply chain network in a negative way, such as causing facility damage or traffic stopage, etc., then Yes. \
            Else if it can potentially disrupt the normal operations a supply chain network, such as social-political disruptions, extreme weathers, or other disruptions, etc., then asnwer Yes. \
            If not or uncertain, such as general knowledge, good news, individual personnel events, project annoucement etc., answer No",
        enum=["Yes", "No"]
    )
    reasoning: str = Field( 
        description="The reasoning behind your impact assessment based on the impact reasoning step above. Explain why you think the event will (Yes) or will not (No) impact the supply chain network."
    )
    vessel_name: Optional[list[str]] = Field(
        default=[""], 
        description="The names of the marine vessels or container ships mentioned in the text, if any."
    )

sample_text = """
Officials involved in the clear up of the bridge collapse at Baltimore Port in the US have said the vessel that remains lodged among debris will be removed by 10 May.
While some ships have been able to navigate in and out of the port through a limited access channel opened up by the coastguard in the middle of the collapsed bridge, the Dali containership has remained in the place where it collided with the Francis Scott Key Bridge since the incident took place at the major port in Maryland on 26 March.
Ahead of the planned removal of the ship, a 35ft deep Fort McHenry Limited Access Channel that had been open for four days and allowed the first container ship to return to the port closed on 29 April, though the three other temporary channels, which are 20, 14 and 11ft deep, will remain open.
Maryland Governor Wes Moore highlighted some of the difficulties that have been faced by the team attempting to clear the bridge debris and Dali vessel.
"""

pydantic_parser = PydanticOutputParser(pydantic_object=NewsInfo)
tools = [NewsInfo]

### 1. Tools and tool binding

- https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models
- https://docs.mistral.ai/capabilities/function_calling/

In [98]:
# Mistral uses the same metadata for tool as OpenAI:
# however, the `tool_choice` forcing is different.
# for OpenAI, use the `tool_choice` as a dictionary with the key `type` and `function` as the value
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
llm_with_tools = llm.bind_tools(tools, tool_choice={"type": "function", "function": {"name": tools[0].__name__}}) # use the class name of the only tool
print(llm_with_tools.kwargs['tools'][0])
print()
mistral_llm = ChatMistralAI(model="mistral-small-2402")
mistral_llm_with_tools = mistral_llm.bind_tools(tools, tool_choice="any")
print(mistral_llm_with_tools.kwargs['tools'][0])

{'type': 'function', 'function': {'name': 'NewsInfo', 'description': 'Information extracted from the text.', 'parameters': {'type': 'object', 'properties': {'title': {'description': 'One sentence summary of the article of maximum 200 characters, prefereably with the event, location and time information.', 'type': 'string'}, 'summary': {'description': 'A short summary of the text, maximum 200 words', 'type': 'string'}, 'impact': {'description': 'Answer only Yes or No to this question: does this event negatively impact a supply chain network (the movement of people and goods)? Answer this by following the following reasoning steps:             If the event can directly impact a supply chain network in a negative way, such as causing facility damage or traffic stopage, etc., then Yes.             Else if it can potentially disrupt the normal operations a supply chain network, such as social-political disruptions, extreme weathers, or other disruptions, etc., then asnwer Yes.             I

#### a. With Direct HumanMessage prompt

In [99]:
query = f"What is the summary, title, impact, reasoning and vessel_name information from the text: {sample_text}"

In [100]:
messages = [HumanMessage(query)]
llm_response = llm_with_tools.invoke(messages)
llm_response.tool_calls[0]['args']

{'title': 'Bridge collapse at Baltimore Port: Vessel to be removed by 10 May',
 'summary': 'Officials involved in the clear up of the bridge collapse at Baltimore Port in the US have said the vessel that remains lodged among debris will be removed by 10 May. Some ships have been able to navigate through a limited access channel opened by the coastguard, but the Dali containership remains in place since the incident on 26 March. Challenges faced in clearing the bridge debris and vessel.',
 'impact': 'Yes',
 'reasoning': 'The bridge collapse and the vessel remaining lodged among debris can disrupt the normal operations of the port and potentially impact the supply chain network by causing delays and hindering the movement of goods and vessels.',
 'vessel_name': ['Dali containership']}

In [101]:
llm_response.__dict__

{'content': '',
 'additional_kwargs': {'tool_calls': [{'id': 'call_hJmOd3l7qL3YO05AhHOS3Hgj',
    'function': {'arguments': '{"title":"Bridge collapse at Baltimore Port: Vessel to be removed by 10 May","summary":"Officials involved in the clear up of the bridge collapse at Baltimore Port in the US have said the vessel that remains lodged among debris will be removed by 10 May. Some ships have been able to navigate through a limited access channel opened by the coastguard, but the Dali containership remains in place since the incident on 26 March. Challenges faced in clearing the bridge debris and vessel.","impact":"Yes","reasoning":"The bridge collapse and the vessel remaining lodged among debris can disrupt the normal operations of the port and potentially impact the supply chain network by causing delays and hindering the movement of goods and vessels.","vessel_name":["Dali containership"]}',
     'name': 'NewsInfo'},
    'type': 'function'}]},
 'response_metadata': {'token_usage': {

In [102]:
mistral_llm_response = mistral_llm_with_tools.invoke(messages)
mistral_llm_response.__dict__

{'content': '',
 'additional_kwargs': {'tool_calls': [{'id': 'Aj4t5vaSK',
    'function': {'name': 'NewsInfo',
     'arguments': '{"title": "Dali Containership Removal Scheduled After Baltimore Port Bridge Collapse", "summary": "The Dali containership, involved in the bridge collapse at Baltimore Port on 26 March, will be removed by 10 May. While some ships have navigated through a limited access channel, the Dali has remained stuck at the collision site. The 35ft deep Fort McHenry Limited Access Channel closed on 29 April, but three other temporary channels remain open. Maryland Governor Wes Moore noted challenges in clearing the bridge debris and the Dali vessel.", "impact": "Yes", "reasoning": "The event directly impacts the supply chain network by causing a facility damage (bridge collapse) and disrupting the normal operations of the port (limited access channels and vessel stuck at the collision site).", "vessel_name": ["Dali"]}'}}]},
 'response_metadata': {'token_usage': {'prompt

In [103]:
mistral_llm_response.tool_calls[0]['args']

{'title': 'Dali Containership Removal Scheduled After Baltimore Port Bridge Collapse',
 'summary': 'The Dali containership, involved in the bridge collapse at Baltimore Port on 26 March, will be removed by 10 May. While some ships have navigated through a limited access channel, the Dali has remained stuck at the collision site. The 35ft deep Fort McHenry Limited Access Channel closed on 29 April, but three other temporary channels remain open. Maryland Governor Wes Moore noted challenges in clearing the bridge debris and the Dali vessel.',
 'impact': 'Yes',
 'reasoning': 'The event directly impacts the supply chain network by causing a facility damage (bridge collapse) and disrupting the normal operations of the port (limited access channels and vessel stuck at the collision site).',
 'vessel_name': ['Dali']}

#### b. With ChatPromptTemplate prompt

In [104]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm, specialized in news analysis."
            "Only extract relevant information from the text.",
        ),
        ("human", "{user_input}"),
    ]
)

In [119]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
llm_with_tools = llm.bind_tools(tools, tool_choice={"type": "function", "function": {"name": tools[0].__name__}})
chain = chat_prompt | llm_with_tools
llm_response = chain.invoke({"user_input": sample_text})
llm_response.__dict__

{'content': '',
 'additional_kwargs': {'tool_calls': [{'id': 'call_ldoO0qUGIYJkRVB6XEiSuFKo',
    'function': {'arguments': '{"title":"Vessel lodged in debris to be removed by 10 May at Baltimore Port after bridge collapse","summary":"Officials at Baltimore Port in the US plan to remove the Dali containership lodged among debris by 10 May following a bridge collapse on 26 March. Limited access channels have been opened for some ships to navigate, with temporary channels remaining open as the cleanup continues.","impact":"Yes","reasoning":"The bridge collapse and the vessel lodged among debris can disrupt the normal operations of the port and the movement of ships, potentially impacting the supply chain network in the area."}',
     'name': 'NewsInfo'},
    'type': 'function'}]},
 'response_metadata': {'token_usage': {'completion_tokens': 118,
   'prompt_tokens': 527,
   'total_tokens': 645},
  'model_name': 'gpt-3.5-turbo-0125',
  'system_fingerprint': None,
  'finish_reason': 'stop',


In [120]:
llm_response.tool_calls[0]['args']

{'title': 'Vessel lodged in debris to be removed by 10 May at Baltimore Port after bridge collapse',
 'summary': 'Officials at Baltimore Port in the US plan to remove the Dali containership lodged among debris by 10 May following a bridge collapse on 26 March. Limited access channels have been opened for some ships to navigate, with temporary channels remaining open as the cleanup continues.',
 'impact': 'Yes',
 'reasoning': 'The bridge collapse and the vessel lodged among debris can disrupt the normal operations of the port and the movement of ships, potentially impacting the supply chain network in the area.'}

In [133]:
final_prompt = chat_prompt.format(user_input=sample_text)
print(f"Tokens from the prompt: {llm_with_tools.get_num_tokens(final_prompt)}")
print(f"Tokens from the tool call: {llm_with_tools.get_num_tokens(str(llm_with_tools.kwargs['tools'][0]['function']))}")

Tokens from the prompt: 222
Tokens from the tool call: 358


In [107]:
chain = chat_prompt | mistral_llm_with_tools
mistral_llm_response = chain.invoke({"user_input": sample_text})
mistral_llm_response.__dict__

{'content': '',
 'additional_kwargs': {'tool_calls': [{'id': 'YzLW7zt5d',
    'function': {'name': 'NewsInfo',
     'arguments': '{"title": "Vessel Stuck in Baltimore Port Bridge Collapse to be Removed by 10 May", "summary": "The Dali containership, stuck in the bridge collapse at Baltimore Port since 26 March, will be removed by 10 May, according to officials. Although some ships have been able to navigate through a limited access channel, the 35ft deep Fort McHenry Limited Access Channel closed on 29 April, while three other temporary channels remain open. Maryland Governor Wes Moore highlighted the challenges faced by the clear-up team.", "impact": "Yes", "reasoning": "The bridge collapse and the stuck vessel directly impact the supply chain network by causing facility damage and limiting the movement of goods through the port. The closure of the 35ft deep channel further exacerbates the situation by restricting the access of larger vessels.", "vessel_name": ["Dali"]}'}}]},
 'respon

In [108]:
mistral_llm_response.tool_calls[0]['args']

{'title': 'Vessel Stuck in Baltimore Port Bridge Collapse to be Removed by 10 May',
 'summary': 'The Dali containership, stuck in the bridge collapse at Baltimore Port since 26 March, will be removed by 10 May, according to officials. Although some ships have been able to navigate through a limited access channel, the 35ft deep Fort McHenry Limited Access Channel closed on 29 April, while three other temporary channels remain open. Maryland Governor Wes Moore highlighted the challenges faced by the clear-up team.',
 'impact': 'Yes',
 'reasoning': 'The bridge collapse and the stuck vessel directly impact the supply chain network by causing facility damage and limiting the movement of goods through the port. The closure of the 35ft deep channel further exacerbates the situation by restricting the access of larger vessels.',
 'vessel_name': ['Dali']}

## 2. JSON-based Instruction Prompting

- For LLMs that do not have native tool-binding support, we can try forcing the LLM to respond to the schema by using the `PydanticOutputParser` class and its `.get_format_instructions()` method.
- This method generates an instruction prompt for the language model on how to format its output so that it conforms to the structure defined by the Pydantic model
- As this is an additional prompt that will be sent to the LLM, it is not strictly considered as a tool-calling method. You will see later that the LLM response reflects this key difference from the previous native tool-calling approach.
- If you compare the prompt_tokens field from the tool-calling method and this method, this method adds significantly to the token count. However, I'm not sure how to accurately attribute this difference by checking the total request sent

In [6]:
# create a PydanticOutputParser object with the schema, this will be used to generate the format instructions, passed to the LLM prompt
from langchain_core.output_parsers.pydantic import PydanticOutputParser
from langchain_core.output_parsers import JsonOutputParser


# translate the Pydantic schema to an output instruction for the LLM
pydantic_parser = PydanticOutputParser(pydantic_object=NewsInfo)
format_instructions = pydantic_parser.get_format_instructions()

# create the prompt with the format instructions
INSTRUCTION_PROMPT_TEMPLATE = """
You are an expert extraction algorithm, specialized in news analysis. Only extract relevant information from the text.
{format_instructions}

news article content:
{news_content}
"""
instruction_prompt = PromptTemplate.from_template(INSTRUCTION_PROMPT_TEMPLATE)

In [7]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
chain = instruction_prompt | llm
llm_response  = chain.invoke({'format_instructions' : format_instructions, 'news_content': sample_text})

In [8]:
# you can see that the output is in the correct format
# but it is in the content field, instead of the tool_call field
# this is because the LLM API treats the request we sent as a single prompt, not a tool call
print(pydantic_parser.parse(llm_response.content).dict())
llm_response.__dict__

{'title': 'Bridge collapse at Baltimore Port in the US', 'summary': 'Officials are planning to remove the Dali containership that remains lodged among debris at Baltimore Port by 10 May. Some ships have been able to navigate through a limited access channel opened by the coastguard, but the Dali vessel has been stuck since the incident on 26 March.', 'impact': 'Yes', 'reasoning': 'The bridge collapse and the lodged vessel are directly impacting the supply chain network at Baltimore Port, causing disruptions and difficulties for ships navigating in and out of the port.', 'vessel_name': ['Dali']}


{'content': '{\n  "title": "Bridge collapse at Baltimore Port in the US",\n  "summary": "Officials are planning to remove the Dali containership that remains lodged among debris at Baltimore Port by 10 May. Some ships have been able to navigate through a limited access channel opened by the coastguard, but the Dali vessel has been stuck since the incident on 26 March.",\n  "impact": "Yes",\n  "reasoning": "The bridge collapse and the lodged vessel are directly impacting the supply chain network at Baltimore Port, causing disruptions and difficulties for ships navigating in and out of the port.",\n  "vessel_name": ["Dali"]\n}',
 'additional_kwargs': {},
 'response_metadata': {'token_usage': {'completion_tokens': 133,
   'prompt_tokens': 723,
   'total_tokens': 856},
  'model_name': 'gpt-3.5-turbo-0125',
  'system_fingerprint': None,
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-e4b697a6-61cb-4301-881b-4ae56d24bbc4-0',
 'example': False,
 'tool

In [9]:
# Looking at the tokens used for instruction, it is higher than the tokens used for tool
# this is because pydantic needed to add other instructions to the prompt to guide the LLM
# while previously, the LLM was sonfigured to use the tool without this additional instruction
# thus the tokens were lower
print(f"Tokens from the instruction: {llm.get_num_tokens(format_instructions)}")

# rough token conciliation below. The difference in tokens is not 100% reconciled because I'm not sure how the tokens are counted
# but it gives a rough idea how the tokens are used in the prompt and the tool call

# previous case:
# Total prompt tokens = 527
# Tokens from the tool call = 358

# current case:
# Total prompt tokens = 723
# Tokens from the tool call = 493

# difference from Total prompt tokens = 723 - 527 = 196
# difference from Tokens from the tool call = 493 - 358 = 135

Tokens from the instruction: 493


In [10]:
# try again with Mistral:
mistral_llm = ChatMistralAI(model="mistral-small-2402")
chain = instruction_prompt | mistral_llm
llm_response  = chain.invoke({'format_instructions' : format_instructions, 'news_content': sample_text})

In [11]:
# the parsed data again is in the content field, instead of the tool_call field
print(pydantic_parser.parse(llm_response.content).dict())
llm_response.__dict__

{'title': 'Dali containership to be removed from Baltimore Port bridge collapse site by 10 May', 'summary': 'The Dali containership, which has been stuck at the site of the bridge collapse at Baltimore Port since 26 March, is set to be removed by 10 May. While some ships have been able to navigate through a limited access channel, the full channel closure has caused disruptions. Maryland Governor Wes Moore has acknowledged the challenges faced by the clear-up team.', 'impact': 'Yes', 'reasoning': 'The bridge collapse and the subsequent lodging of the Dali containership have directly impacted the supply chain network by causing facility damage and disrupting the normal operations at Baltimore Port. The closure of the full access channel has limited the movement of goods and vessels, causing delays and disruptions.', 'vessel_name': ['Dali']}


{'content': '{\n  "title": "Dali containership to be removed from Baltimore Port bridge collapse site by 10 May",\n  "summary": "The Dali containership, which has been stuck at the site of the bridge collapse at Baltimore Port since 26 March, is set to be removed by 10 May. While some ships have been able to navigate through a limited access channel, the full channel closure has caused disruptions. Maryland Governor Wes Moore has acknowledged the challenges faced by the clear-up team.",\n  "impact": "Yes",\n  "reasoning": "The bridge collapse and the subsequent lodging of the Dali containership have directly impacted the supply chain network by causing facility damage and disrupting the normal operations at Baltimore Port. The closure of the full access channel has limited the movement of goods and vessels, causing delays and disruptions.",\n  "vessel_name": ["Dali"]\n}',
 'additional_kwargs': {},
 'response_metadata': {'token_usage': {'prompt_tokens': 785,
   'total_tokens': 985,
   '

In [12]:
# try again with Mistral 7b
mistral_llm = ChatMistralAI(model="open-mistral-7b")
chain = instruction_prompt | mistral_llm
llm_response  = chain.invoke({'format_instructions' : format_instructions, 'news_content': sample_text})
print(pydantic_parser.parse(llm_response.content).dict())
llm_response.__dict__

{'title': 'Bridge collapse at Baltimore Port impacting vessel traffic', 'summary': 'The collapse of a bridge at Baltimore Port in the US has resulted in a vessel, the Dali containership, being lodged amidst debris. Officals have planned to remove the vessel by 10 May, but the closure of a deep channel has caused some disruptions. Maryland Governor Wes Moore has acknowledged the challenges faced in clearing the bridge and vessel.', 'impact': 'Yes', 'reasoning': 'The bridge collapse at Baltimore Port has caused a vessel to become lodged, disrupting normal shipping operations. The closure of a deep channel for four days has further impacted shipping, as it allowed only one container ship to return to the port. Maryland Governor Wes Moore has acknowledged the difficulties faced in clearing the bridge debris and vessel, indicating a potential ongoing impact.', 'vessel_name': ['Dali containership']}


{'content': '```\n{\n  "title": "Bridge collapse at Baltimore Port impacting vessel traffic",\n  "summary": "The collapse of a bridge at Baltimore Port in the US has resulted in a vessel, the Dali containership, being lodged amidst debris. Officals have planned to remove the vessel by 10 May, but the closure of a deep channel has caused some disruptions. Maryland Governor Wes Moore has acknowledged the challenges faced in clearing the bridge and vessel.",\n  "impact": "Yes",\n  "reasoning": "The bridge collapse at Baltimore Port has caused a vessel to become lodged, disrupting normal shipping operations. The closure of a deep channel for four days has further impacted shipping, as it allowed only one container ship to return to the port. Maryland Governor Wes Moore has acknowledged the difficulties faced in clearing the bridge debris and vessel, indicating a potential ongoing impact.",\n  "vessel_name": ["Dali containership"]\n}\n```',
 'additional_kwargs': {},
 'response_metadata': {'

In [13]:
# try again with snowflake-hosted LLM:
# the snowflake LLM response is not the AIMessage object but a string with the parsed data and other metadata
# so we use JsonOutputParser to parse the response for easy access to the parsed data
llm = SnowflakeCortexLLM(sp_session = sp_session, model='mistral-7b')
chain = instruction_prompt | llm | JsonOutputParser()
llm_response  = chain.invoke({'format_instructions' : format_instructions, 'news_content': sample_text})

In [14]:
result = llm_response['choices'][0]['messages']
print(pydantic_parser.parse(result).dict())
llm_response

{'title': 'Baltimore Port Bridge Collapse: Dali Containship Removal Delayed, Channel Closed', 'summary': 'The Dali containership, which collided with the Francis Scott Key Bridge at Baltimore Port in the US on 26 March, remains lodged among the debris and will not be removed until 10 May. A 35ft deep channel that had been open for four days, allowing the first container ship to return to the port, was closed on 29 April. Maryland Governor Wes Moore discussed the challenges faced by the team clearing the bridge debris and the vessel.', 'impact': 'Yes', 'reasoning': 'The bridge collapse at Baltimore Port has caused the Dali containership to remain lodged among the debris, preventing it from being removed until 10 May. The closure of the 35ft deep channel for repairs further disrupts the normal operations of the supply chain network by limiting the size and number of vessels that can access the port. These disruptions can potentially cause delays and increased costs for businesses and ind

{'choices': [{'messages': ' {\n"title": "Baltimore Port Bridge Collapse: Dali Containship Removal Delayed, Channel Closed",\n"summary": "The Dali containership, which collided with the Francis Scott Key Bridge at Baltimore Port in the US on 26 March, remains lodged among the debris and will not be removed until 10 May. A 35ft deep channel that had been open for four days, allowing the first container ship to return to the port, was closed on 29 April. Maryland Governor Wes Moore discussed the challenges faced by the team clearing the bridge debris and the vessel.",\n"impact": "Yes",\n"reasoning": "The bridge collapse at Baltimore Port has caused the Dali containership to remain lodged among the debris, preventing it from being removed until 10 May. The closure of the 35ft deep channel for repairs further disrupts the normal operations of the supply chain network by limiting the size and number of vessels that can access the port. These disruptions can potentially cause delays and incre

## 3. For Custom LLM without tool binding support

- This is not an "official" method to invoke tool-calling API of a LLM behind the scene.
- When tried with the original pydantic model, the results have missing key if the key is marked as Optional or with a default value inside the Pydantic model.
- The behavior above is true for both Custom LLM derived from snowflake, and even for native OpenAI binding
- Attempt to modify the original Pydantic schema worsened the situation even for OpenAI models using rendered_tools derived from its own bind_tools() function. 
- Maybe the `llm_with_tools.kwargs['tools'][0]['function']` method to get the parsed tool is not suitable to be passed back to OpenAI.
- The behabior is similar when Snowflake models were tested.
- Thus this is not purused further.

In [100]:
# use OpenAI class to act as a helper to render the tools from the Pydantic schema
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
tools = [NewsInfo]
llm_with_tools = llm.bind_tools(tools)
rendered_tools = llm_with_tools.kwargs['tools'][0]['function']

# remove/escape any single quotes from the rendered tools and the sample text
# as they will interfere with the Snowflake SQL statement
rendered_tools_json = json.dumps(rendered_tools)
sample_text_cleaned = sample_text.replace("'", "\\'")

In [86]:
from langchain_core.prompts import ChatPromptTemplate

# note that the prompt must also be free from single quotes
chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm, specialized in news analysis."
            "You have access to the following set of tools: {rendered_tools}"
        ),
        ("human", "{user_input}"),
    ]
)


In [107]:
sf_llm = SnowflakeCortexLLM(sp_session=sp_session, model='mistral-7b')
chain = chat_prompt | sf_llm | JsonOutputParser() # better than StrOutputParser() for JSON output
llm_response = chain.invoke({"rendered_tools":rendered_tools_json, "user_input": sample_text_cleaned})
llm_response

{'choices': [{'messages': ' {\n  "title": "Maryland\'s Baltimore Port: Dali Containship Remains Lodged Among Bridge Debris, Affecting Navigation and Closing Access Channels",\n  "summary": "The Dali containership has been stuck at the Baltimore Port in the US since colliding with the Francis Scott Key Bridge on 26 March. The incident caused the closure of a 35ft deep access channel, affecting the navigation of larger vessels. The channel reopened briefly to allow one ship to pass, but was closed again on 29 April. The removal of the Dali vessel is planned for 10 May, but the incident has caused difficulties for the clear-up team, as mentioned by Maryland Governor Wes Moore.",\n  "impact": "Yes",\n  "reasoning": "The bridge collapse and the subsequent lodging of the Dali containership in the debris have directly impacted the supply chain network at the Baltimore Port by closing a major access channel for larger vessels. The closure has prevented these vessels from entering or leaving th

In [111]:
eval(llm_response['choices'][0]['messages'])

{'title': "Maryland's Baltimore Port: Dali Containship Remains Lodged Among Bridge Debris, Affecting Navigation and Closing Access Channels",
 'summary': 'The Dali containership has been stuck at the Baltimore Port in the US since colliding with the Francis Scott Key Bridge on 26 March. The incident caused the closure of a 35ft deep access channel, affecting the navigation of larger vessels. The channel reopened briefly to allow one ship to pass, but was closed again on 29 April. The removal of the Dali vessel is planned for 10 May, but the incident has caused difficulties for the clear-up team, as mentioned by Maryland Governor Wes Moore.',
 'impact': 'Yes',
 'reasoning': "The bridge collapse and the subsequent lodging of the Dali containership in the debris have directly impacted the supply chain network at the Baltimore Port by closing a major access channel for larger vessels. The closure has prevented these vessels from entering or leaving the port, causing potential delays and di

In [114]:
sf_llm = SnowflakeCortexLLM(sp_session=sp_session, model='mixtral-8x7b')
chain = chat_prompt | sf_llm | JsonOutputParser() # better than StrOutputParser() for JSON output
llm_response = chain.invoke({"rendered_tools":rendered_tools_json, "user_input": body_cleaned[-1]})
llm_response

{'choices': [{'messages': ' NewsInfo:\n{\n  "title": "Bulk carrier CRISTIN HT suffers engine failure, towed to Aliaga, Turkey; General Average Salvage possible",\n  "summary": "The Bulk carrier CRISTIN HT (IMO:9455985) experienced an engine failure approximately 43 nautical miles southeast of Syracuse, Italy, on 15 May 2024. Unable to complete repairs, the vessel was taken under tow to Aliaga, Turkey. This incident raises the possibility of General Average Salvage and associated recovery issues.",\n  "impact": "Yes",\n  "reasoning": "The engine failure and subsequent towing of the CRISTIN HT directly impact the vessel\'s ability to transport goods, causing a disruption in the supply chain network. This event may lead to financial losses, delays, and potential cargo damage, affecting the stakeholders involved.",\n  "vessel_name": ["CRISTIN HT"]\n}'}],
 'created': 1718166230,
 'model': 'mixtral-8x7b',
 'usage': {'completion_tokens': 219,
  'prompt_tokens': 567,
  'total_tokens': 786}}

In [116]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
chain = chat_prompt | llm | JsonOutputParser()
llm_response = chain.invoke({"rendered_tools":rendered_tools_json, "user_input": body_cleaned[-1]})
llm_response

{'title': 'Bulk carrier CRISTIN HT suffers engine failure near Syracuse, Italy and is towed to Aliaga, Turkey',
 'summary': 'Bulk carrier CRISTIN HT (IMO:9455985) experienced an engine failure approximately 43 nautical miles southeast of Syracuse, Italy on 15 May 2024. The crew was unable to repair the issue, leading to the vessel being towed to Aliaga, Turkey. General Average Salvage and associated Recovery issues may arise from this incident, prompting a call for concerned parties to protect their interests.',
 'impact': 'Yes',
 'reasoning': 'The engine failure of the bulk carrier CRISTIN HT and its subsequent tow to Aliaga, Turkey can disrupt the normal operations of the supply chain network. The potential General Average Salvage and Recovery issues could further impact the movement of goods and vessels in the area, making this event a negative impact on the supply chain network.'}

In [123]:
class NewsInfoV2(BaseModel):
    """Information extracted from the text."""
    title: str = Field(
        description="One sentence summary of the article of maximum 200 characters, prefereably with the event, location and time information."
    )
    summary: str = Field(
        description="A short summary of the text, maximum 200 words"
    )
    impact: str = Field( 
        description="Answer only Yes or No to this question: does this event negatively impact a supply chain network (the movement of people and goods)? Answer this by following the following reasoning steps: \
            If the event can directly impact a supply chain network in a negative way, such as causing facility damage or traffic stopage, etc., then Yes. \
            Else if it can potentially disrupt the normal operations a supply chain network, such as social-political disruptions, extreme weathers, or other disruptions, etc., then asnwer Yes. \
            If not or uncertain, such as general knowledge, good news, individual personnel events, project annoucement etc., answer No",
        enum=["Yes", "No"]
    )
    reasoning: str = Field( 
        description="The reasoning behind your impact assessment based on the impact reasoning step above. Explain why you think the event will (Yes) or will not (No) impact the supply chain network."
    )
    vessel_name: list[str] = Field(
        description="The names of the marine vessels or container ships mentioned in the text, if any."
    )

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
tools = [NewsInfoV2]
llm_with_tools = llm.bind_tools(tools)
rendered_tools = llm_with_tools.kwargs['tools'][0]['function']
rendered_tools_json = json.dumps(rendered_tools)

In [124]:
rendered_tools_json

'{"name": "NewsInfoV2", "description": "Information extracted from the text.", "parameters": {"type": "object", "properties": {"title": {"description": "One sentence summary of the article of maximum 200 characters, prefereably with the event, location and time information.", "type": "string"}, "summary": {"description": "A short summary of the text, maximum 200 words", "type": "string"}, "impact": {"description": "Answer only Yes or No to this question: does this event negatively impact a supply chain network (the movement of people and goods)? Answer this by following the following reasoning steps:             If the event can directly impact a supply chain network in a negative way, such as causing facility damage or traffic stopage, etc., then Yes.             Else if it can potentially disrupt the normal operations a supply chain network, such as social-political disruptions, extreme weathers, or other disruptions, etc., then asnwer Yes.             If not or uncertain, such as ge

In [130]:
# however, the Snowflake Cortex LLM and the OpenAI LLM were not able to parse the optional field correctlye
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
chain = chat_prompt | llm | JsonOutputParser()
llm_response = chain.invoke({"rendered_tools":rendered_tools_json, "user_input": body_cleaned[-1]})
llm_response

{'name': 'NewsInfoV2',
 'description': 'Information extracted from the text.',
 'parameters': {'type': 'object',
  'properties': {'title': {'description': 'Bulk carrier CRISTIN HT suffers engine failure near Syracuse, Italy and is towed to Aliaga, Turkey',
    'type': 'string'},
   'summary': {'description': 'Bulk carrier CRISTIN HT experienced an engine failure near Syracuse, Italy and was towed to Aliaga, Turkey. General Average Salvage and Recovery issues may arise.',
    'type': 'string'},
   'impact': {'description': 'Yes', 'enum': ['Yes', 'No'], 'type': 'string'},
   'reasoning': {'description': 'The engine failure and towing of the vessel can disrupt the normal operations of the supply chain network, potentially leading to delays and additional costs. General Average Salvage and Recovery issues may further impact the cargo owners and stakeholders.',
    'type': 'string'},
   'vessel_name': {'description': ['CRISTIN HT'],
    'type': 'array',
    'items': {'type': 'string'}}},
  

### 2. With Pydantic formatted instructions

In [146]:
pydantic_parser = PydanticOutputParser(pydantic_object=NewsInfo)
format_instructions = pydantic_parser.get_format_instructions()

In [147]:
# the format_instructions is just a string that contains the Pydantic schema pasrsed to JSON, 
# with some other instructions and examples,
# check the source code for more details
print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Information extracted from the text.", "properties": {"title": {"title": "Title", "description": "One sentence summary of the article of maximum 200 characters, prefereably with the event, location and time information.", "type": "string"}, "summary": {"title": "Summary", "description": "A short summary of the text, maximum 200 words", "type": "string"}, "impact": {"title": "Impact", "description": "Answer only Yes or No to this question: does this event negatively impact a supply chain network (the movement of people and good

In [141]:
prompt_instruct = PromptTemplate.from_template(PROMPT_TEMPLATE_INSTRUCT)
sf_llm = SnowflakeCortexLLM(sp_session = sp_session, model='mistral-7b')
chain = prompt_instruct | sf_llm | JsonOutputParser() 

In [142]:
llm_response  = chain.invoke({'format_instructions' : format_instructions, 'news_content': sample_text})

In [145]:
eval(llm_response['choices'][0]['messages'])

{'title': 'Baltimore Port Bridge Collapse: Dali Containship Removal Delayed, Channel Closed',
 'summary': 'Officials have announced that the Dali containership, which collided with the Francis Scott Key Bridge at Baltimore Port in March, will be removed by 10 May. However, a deep access channel that had been open for four days, allowing the first container ship to return to the port, was closed on 29 April. Maryland Governor Wes Moore discussed the challenges faced by the team clearing the bridge debris and the vessel.',
 'impact': 'Yes',
 'reasoning': 'The bridge collapse at Baltimore Port has directly impacted the supply chain network by preventing the Dali containership from being removed, causing a closure of a deep access channel that had allowed the first container ship to return to the port. This disruption to the normal operations of the supply chain network is expected to continue until the Dali containership is removed and the channel is reopened.',
 'vessel_name': ['Dali', '

In [149]:
llm_response 

{'choices': [{'messages': ' {\n"title": "Baltimore Port Bridge Collapse: Dali Containship Removal Delayed, Channel Closed",\n"summary": "Officials have announced that the Dali containership, which collided with the Francis Scott Key Bridge at Baltimore Port in March, will be removed by 10 May. However, a deep access channel that had been open for four days, allowing the first container ship to return to the port, was closed on 29 April. Maryland Governor Wes Moore discussed the challenges faced by the team clearing the bridge debris and the vessel.",\n"impact": "Yes",\n"reasoning": "The bridge collapse at Baltimore Port has directly impacted the supply chain network by preventing the Dali containership from being removed, causing a closure of a deep access channel that had allowed the first container ship to return to the port. This disruption to the normal operations of the supply chain network is expected to continue until the Dali containership is removed and the channel is reopened